In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
# Function definitions

def payload_i (x,y,beta_i):
    Hi=np.exp(-beta_i *y/x) -x**2 *(1-np.exp(-y/x))
    return Hi


def payload_mission(x,y,sequence):
    H_list=[]
    for beta_i in sequence:
        H_list.append(payload_i(x,y,beta_i))
    Hm=np.min(H_list)
    return Hm


def time_coeff (x,y,sequence):
    mp_refuel= np.sum(1-np.exp(-sequence *y/x))
    mp_star=1-np.exp(-y/x)
    tau=mp_refuel/mp_star
    return tau


def specific_fuel_cons(x,y,sequence):
    mp_refuel= np.sum(1-np.exp(-sequence *y/x))
    Hm=payload_mission(x,y,sequence)
    f=mp_refuel/Hm
    return f

In [3]:
# Constraint functions

def constraint_sum_beta(sequence):
    return np.sum(sequence) - 1

def constraint_min_beta(sequence):
    return np.min(sequence) - 0.05

def constraint_payload_min(sequence, x, y, H_min):
    return payload_mission(x, y, sequence) - H_min

def constraint_tau_max(sequence, x, y, tau_max):
    return tau_max - time_coeff(x, y, sequence)

def constraint_f_max(sequence, x, y, f_max):
    return f_max - specific_fuel_cons(x, y, sequence)

In [4]:
# Initial guess and bounds
x = 1.2
y = 0.5
H_max=1 -x**2 *(1-np.exp(-y/x))
H_min = 0.80*H_max
tau_max = 2
f_max = 2.5
n = 10
beta_initial = np.full(n, 1/n)
bounds = [(0, 1)] * n

# Constraints with parameters
constraints = [
    {'type': 'eq', 'fun': constraint_sum_beta},
    {'type': 'ineq', 'fun': constraint_min_beta},
    {'type': 'ineq', 'fun': lambda beta: constraint_payload_min(beta, x, y, H_min)},
    {'type': 'ineq', 'fun': lambda beta: constraint_tau_max(beta, x, y, tau_max)},
    {'type': 'ineq', 'fun': lambda beta: constraint_f_max(beta, x, y, f_max)},
]

# 1- Maximize payload mass

In [5]:
result = minimize(
    lambda beta: -payload_mission(x, y, beta),
    beta_initial,
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'maxiter': 1000, 'disp': True}
)

# Results
if result.success:
    print(f"Optimal sequence: {[f'{beta:.3f}' for beta in result.x]}")
    print(f"Payload mission: {payload_mission(x, y, result.x):.3f}")
    print(f"Time coeff: {time_coeff(x, y, result.x):.3f}")
    print(f"Specific fuel cons: {specific_fuel_cons(x, y, result.x):.3f}")
else:
    print("Optimization failed:", result.message)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.4684959645977772
            Iterations: 1
            Function evaluations: 11
            Gradient evaluations: 1
Optimal sequence: ['0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100']
Payload mission: 0.468
Time coeff: 1.198
Specific fuel cons: 0.871


# 2- Minimize the time ratio

In [6]:
result = minimize(
    lambda beta: time_coeff(x, y, beta),
    beta_initial,
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'maxiter': 1000, 'disp': True}
)

# Results
if result.success:
    print(f"Optimal sequence: {[f'{beta:.3f}' for beta in result.x]}")
    print(f"Payload mission: {payload_mission(x, y, result.x):.3f}")
    print(f"Time coeff: {time_coeff(x, y, result.x):.3f}")
    print(f"Specific fuel cons: {specific_fuel_cons(x, y, result.x):.3f}")
else:
    print("Optimization failed:", result.message)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1976352378767365
            Iterations: 1
            Function evaluations: 11
            Gradient evaluations: 1
Optimal sequence: ['0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100']
Payload mission: 0.468
Time coeff: 1.198
Specific fuel cons: 0.871


# 3- Minimize specific propellant consumption

In [7]:
result = minimize(
    lambda beta: specific_fuel_cons(x, y, beta),
    beta_initial,
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'maxiter': 1000, 'disp': True}
)

# Results
if result.success:
    print(f"Optimal sequence: {[f'{beta:.3f}' for beta in result.x]}")
    print(f"Payload mission: {payload_mission(x, y, result.x):.3f}")
    print(f"Time coeff: {time_coeff(x, y, result.x):.3f}")
    print(f"Specific fuel cons: {specific_fuel_cons(x, y, result.x):.3f}")
else:
    print("Optimization failed:", result.message)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.8710969992217402
            Iterations: 1
            Function evaluations: 11
            Gradient evaluations: 1
Optimal sequence: ['0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100', '0.100']
Payload mission: 0.468
Time coeff: 1.198
Specific fuel cons: 0.871


# Plots

In [8]:
# Initial guess and bounds
x = 1.2
y = 0.5
H_max=1 -x**2 *(1-np.exp(-y/x))
H_min = 0.80*H_max
tau_max = 20
f_max = 20
n = 10
np.seed=437
beta_initial = np.full(n, 1/n)
bounds = [(0, 1)] * n

# Constraints with parameters
constraints = [
    {'type': 'eq', 'fun': constraint_sum_beta},
    {'type': 'ineq', 'fun': constraint_min_beta},
    {'type': 'ineq', 'fun': lambda beta: constraint_payload_min(beta, x, y, H_min)},
    {'type': 'ineq', 'fun': lambda beta: constraint_tau_max(beta, x, y, tau_max)},
    {'type': 'ineq', 'fun': lambda beta: constraint_f_max(beta, x, y, f_max)},
]

In [9]:
print("H_min=",H_min)
print(payload_mission(x, y, beta_initial))
print(time_coeff(x, y, beta_initial))
print(specific_fuel_cons(x, y, beta_initial))

H_min= 0.4074452059909113
0.4684959645977772
1.1976352378767365
0.8710969992217402


In [14]:
x = 1.2
y = 0.5
H_max=1 -x**2 *(1-np.exp(-y/x))
H_min = 0.80*H_max
tau_max = 2
f_max = 2
n = 10
np.seed=437
perturbation = np.random.uniform(-0.2, 0.02, n)
beta_initial = np.full(n, 1/n) + perturbation

# Ensure the sum is still 1 and adjust for feasibility
beta_initial /= beta_initial.sum()

bounds = [(0, 1)] * n

# Constraints with parameters
constraints = [
    {'type': 'eq', 'fun': constraint_sum_beta},
    {'type': 'ineq', 'fun': constraint_min_beta},
    {'type': 'ineq', 'fun': lambda beta: constraint_payload_min(beta, x, y, H_min)},
    {'type': 'ineq', 'fun': lambda beta: constraint_tau_max(beta, x, y, tau_max)},
    {'type': 'ineq', 'fun': lambda beta: constraint_f_max(beta, x, y, f_max)},
]


result = minimize(
    lambda beta: time_coeff(x, y, beta),
    beta_initial,
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'maxiter': 1000, 'disp': True}
)

# Results
if result.success:
    print(f"Optimal sequence: {[f'{beta:.3f}' for beta in result.x]}")
    print(f"Payload mission: {payload_mission(x, y, result.x):.3f}")
    print(f"Time coeff: {time_coeff(x, y, result.x):.3f}")
    print(f"Specific fuel cons: {specific_fuel_cons(x, y, result.x):.3f}")
else:
    print("Optimization failed:", result.message)

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 2.010891295483427
            Iterations: 26
            Function evaluations: 253
            Gradient evaluations: 22
Optimization failed: Positive directional derivative for linesearch


In [11]:
H_min

0.4074452059909113

In [12]:
print(f"Payload mission: {payload_mission(x, y, beta_initial):.3f}")
print(f"Time coeff: {time_coeff(x, y, beta_initial):.3f}")
print(f"Specific fuel cons: {specific_fuel_cons(x, y, beta_initial):.3f}")

Payload mission: 0.301
Time coeff: 1.102
Specific fuel cons: 1.248
